In [3]:
# Ce script permet de trouver les mots les plus importants pour décrire les clusters de métiers trouvés
# lors du clustering

# Prérequis :
# - un fichier jobs_clusters.csv qui contient les clusters et les définitions des métiers du cluster
# (fichier obtenu à l'aide du script tfidf_data_preprocessing)

import os
import json
import pandas as pd

from nltk import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import math

In [4]:
# Préparation des chemins utiles
dir_path = os.getcwd()
metiers_dir_path = os.path.join(dir_path, 'jobs_cluster.json')
tf_idf_file = os.path.join(dir_path,'tf_idf.json')

In [5]:
with open(metiers_dir_path, 'r') as f:  # with statement ensure file closing even if an error occure
    jobs_cluster = json.load(f)

nb_clus = len(jobs_cluster.keys())

In [6]:
# Stemmer
stemmer = SnowballStemmer('french')

In [7]:
# Fonction pour nettoyer le texte des caractères spéciaux
special_chars = [",", ".", "'", ";", "\\n",'"',")","(","»","«"]

def clean_text(text, characters):
    c_text = text
    for char in characters:
        c_text = c_text.replace(char, " ")
    c_text = c_text.lower()
    return(c_text)

def tokenize(text, characters):
    c_text = clean_text(text, characters)
    text_tokens = c_text.split(" ")
    return(text_tokens)

In [8]:
# Nettoyage de la ponctuation dans les définitions des métiers
jobs_cluster_pool = {}
for k, desc in jobs_cluster.items():
    desc_str = str(desc)
    desc_c = clean_text(desc_str, special_chars)
    jobs_cluster_pool[k]=desc_c

In [9]:
# Calcul de document frequency
df = {}
for desc in jobs_cluster_pool.values():
    for token in set(word_tokenize(desc)):
        stem = stemmer.stem(token)
        if stem in df:
            df[stem] += 1
        else:
            df[stem] = 1

In [10]:
# Récupération des mots stemmés
stem_dict = {}
for k, desc in jobs_cluster_pool.items():
    tokens = word_tokenize(desc)
    for t in tokens:
        token_stem = stemmer.stem(t)
        if token_stem in stem_dict:
            stem_dict[token_stem].append(t)
        else:
            stem_dict[token_stem] = [t]

for k, v in stem_dict.items():
    stem_dict[k] = list(set(stem_dict[k]))

In [11]:
# tf-idf
tf_idf ={}
for k, desc in jobs_cluster_pool.items():
    tf_idf_clus = {}
    tokens = word_tokenize(desc)
    tokens = [stemmer.stem(t) for t in tokens]
    frequencies = FreqDist(tokens)

    for token, freq in frequencies.items():
        tf_idf_clus[token] = freq * math.log( nb_clus / (1 + df.get(token, 0)) )

    tf_idf[k] = tf_idf_clus

In [12]:
# Tri des mots par leur valeur de tf-idf.
# On peut choisir le nombre de mot final à conserver à la dernière ligne (ici 15 mots)
sorted_tfidf = {}
for clus, dico in tf_idf.items():
    list_temp = [(j, i) for i, j in dico.items()]  # on met la valeur du tfidf du mot en 1er attribut
    list_temp = sorted(list_temp, reverse=True)  # on ordonne la liste suivant les valeurs de tfidf décroissante
    tfidf_clus = [(j,i) for i, j in list_temp]
    sorted_clus = []
    for (token, j) in tfidf_clus:
        token = stem_dict[token]
        sorted_clus.append([token, j])
    sorted_tfidf[clus] = sorted_clus[0:15]

In [13]:
final_tfidf = {}
for k, v in sorted_tfidf.items():
    final_tfidf[k] = []
    for item in v:
        final_tfidf[k].append(item[0][0])

In [14]:
# Copie du dictionnaire au format JSON
with open(tf_idf_file, 'w') as f:
    json.dump(final_tfidf, f, indent=4)

In [15]:
final_tfidf

{'1': ['forestières',
  'récolte',
  'environnementale',
  'élagage',
  'sylvicole',
  'plantation',
  'viticole',
  'cultures',
  'arbres',
  'arboricoles',
  'préserver',
  'abattage',
  'voies',
  'rend',
  'normés'],
 '10': ['sculptures',
  'marqueterie',
  'bois',
  'massifs',
  'protection',
  'mobiliers',
  'restaurer',
  'décors',
  'petits',
  'objets',
  'unité',
  'sculpter',
  'rustique',
  'pavés',
  'imposé'],
 '100': ['touristes',
  'personnes',
  'routier',
  'circuit',
  'biens',
  'transporte',
  'variée',
  'télésiège',
  'téléphérique',
  'télécabines',
  'téléski',
  'skiables',
  'ski',
  'moins',
  'funiculaire'],
 '11': ['décors',
  'anciens',
  'transmettre',
  'savoir-faire',
  'pièce',
  'utilitaires',
  'profession',
  'objets',
  'sensibiliser',
  'restaurer',
  'petits',
  'instrument',
  'métal',
  'façonne',
  'série'],
 '12': ['bois',
  'agence',
  'fermetures',
  'fenêtres',
  'préfabriqués',
  'cloisons',
  'finalités',
  'bâtir',
  'poses',
  'montag